## Search API (Python version)
This notebook explains how to use the API to search the database using either search terms or a set of bibcodes. Lists of query parameters and search fields are also given. Examples here will be shown using Python with the use of the [requests](http://docs.python-requests.org/en/master/) library, though the same work could be done using the [unofficial Python ADS library](https://ads.readthedocs.io/en/latest/) or curl commands on the command line (see the "API documentation - UNIX shell" folder in this same repository).

The **base_url** for queries is `https://api.adsabs.harvard.edu/v1/search`. The given endpoint in each section should be appended to the end of the base URL to make the full URL to be used in the request. 

In all examples below, `token` should be replaced with [your own API token](https://ui.adsabs.harvard.edu/user/settings/token). If you haven't worked with our API before, it's recommended that you read the [README](https://github.com/adsabs/adsabs-dev-api/blob/master/README.md) before beginning.

In [1]:
# import the requests package and set your token in a variable for later use
import requests

token="your-token-here"

### Contents
- Search API
- Query parameters
- Fields
- Examples
- Advanced search syntax

### Get search results
URL: `{base_url}/query?` + query parameters

Method: `GET`

To search, construct a URL using the URL given above, along with your query parameters as detailed in the Query Parameters section below. Multiple search parameters may be added to the URL; separate parameters using an ampersand (`&`). All text must be UTF-8 and URL encoded. For example:

In [2]:
from urllib.parse import urlencode, quote_plus

# accented letters, special characters, and spaces need to be encoded
query = {"q": "author:martínez neutron star"}

encoded_query = urlencode(query)
print(encoded_query)

# note that the colon (:) may be encoded, depending on the algorithm you use. Your request 
# should accept either the unencoded colon (:) or the encoded version (%3A)

q=author%3Amart%C3%ADnez+neutron+star


The response from your request will be JSON encoded. 

Also, note that the search API uses the same syntax as [Apache Solr](http://lucene.apache.org/solr/). For a full reference of query possibilities, please refer to the Solr documentation and [ADS Search Help](https://ui.adsabs.harvard.edu/help/search/). Later sections in this notebook present useful parameters and patterns for the vast majority of use cases, but are not meant to be exhaustive.

In [3]:
encoded_query = urlencode({"q": "author:martínez neutron star",
                           "fl": "title, bibcode",
                           "rows": 1
                          })
results = requests.get("https://api.adsabs.harvard.edu/v1/search/query?{}".format(encoded_query), \
                       headers={'Authorization': 'Bearer ' + token})

# format the response in a nicely readable format
results.json()

{'responseHeader': {'status': 0,
  'QTime': 2140,
  'params': {'q': 'author:martínez neutron star',
   'fl': 'title,bibcode',
   'start': '0',
   'internal_logging_params': 'X-Amzn-Trace-Id=Root=1-63dbf303-455e6a527203ab7166b505b1',
   'rows': '1',
   'wt': 'json'}},
 'response': {'numFound': 299,
  'start': 0,
  'docs': [{'bibcode': '2017ApJ...848L..12A',
    'title': ['Multi-messenger Observations of a Binary Neutron Star Merger']}]}}

### Parse a query
URL: `{base_url}/qtree?` + query parameters

Method: `GET`

Use the syntax above to return a query tree, also known as an Abstract Syntax Tree (AST), as understood by our query parser. This can be useful if you want to modify and/or enhance queries.

In [4]:
encoded_query = urlencode({"q": "author:martínez neutron star"})
results = requests.get("https://api.adsabs.harvard.edu/v1/search/qtree?{}".format(encoded_query), \
                       headers={'Authorization': 'Bearer ' + token})

results.json()

{'responseHeader': {'status': 0, 'QTime': 1},
 'qtree': '\n{"name":"OPERATOR", "label":"DEFOP", "children": [\n    {"name":"MODIFIER", "label":"MODIFIER", "children": [\n        {"name":"TMODIFIER", "label":"TMODIFIER", "children": [\n            {"name":"FIELD", "label":"FIELD", "children": [\n                {"name":"TERM_NORMAL", "input":"author", "start":0, "end":5},\n                {"name":"QNORMAL", "label":"QNORMAL", "children": [\n                    {"name":"TERM_NORMAL", "input":"martínez", "start":7, "end":14}]\n                }]\n            }]\n        }]\n    },\n    {"name":"MODIFIER", "label":"MODIFIER", "children": [\n        {"name":"TMODIFIER", "label":"TMODIFIER", "children": [\n            {"name":"FIELD", "label":"FIELD", "children": [\n                {"name":"QNORMAL", "label":"QNORMAL", "children": [\n                    {"name":"TERM_NORMAL", "input":"neutron", "start":16, "end":22}]\n                }]\n            }]\n        }]\n    },\n    {"name":"MODIF

### Post a large identifier query
URL: `{base_url}/bigquery?` + query parameters

Method: `POST` (note that a `payload` is required for a POST request; see table below for more information)


This type of query, typically called a "big query" for short, returns standard search results, but instead of the typical search terms, accepts as its input a list of bibcodes. There is a maximum of 2000 bibcodes allowed for each big query request. There are also limits on how often you can make a big query request; typically, only 100 requests per day per user are allowed.

Name          | Type   | Description 
:-------------|:------:|:--------------
query parameters   |`string`| **Required**. Query parameters must be encoded as in a standard query. The query parameter `q=` must be included even if no additional searching is desired; to prevent it from restricting the search results, enter `q=*:*`. It's also recommended to use the `fl=` parameter to specify which fields to return.
`payload`     |`string`| **Required**. Newline (`\n`) separated list of bibcodes to return. The first line specifies the field that will be used for searching, i.e. `bibcode` (for example, `bibcode\n1907AN....174...59.\n1908PA.....16..445.\n1989LNP...334..242S`)

Notes:
- Currently, we only allow searching in the `bibcode` field. You can submit both canonical bibcodes (the ones that appear on a paper's abstract page, or in the URL pointing to the abstract page) as well as alternate bibcodes (e.g. bibcodes that pointed to a previous version of a paper, such as the arXiv version). The search automatically recognizes both types.
- The list of bibcodes supplied in the `payload` is *only applied after the main search*, specified with the `q=` parameter. To return the entire list of supplied bibcodes, use `q=*:*`.
- Currently, the default number of results returned by the big query is not controlled by the number of bibcodes requested, but rather is set to the default value of 10. To adjust this, set `&rows=` either to the number of bibcodes requested or to the maximum value of 2000.


In [5]:
encoded_query = urlencode({"q": "*:*",
                           "fl": "bibcode,title",
                           "rows": 2000
                          })
payload = "bibcode\n1907AN....174...59.\n1908PA.....16..445.\n1989LNP...334..242S"
results = requests.post("https://api.adsabs.harvard.edu/v1/search/bigquery?{}".format(encoded_query), \
                       headers={'Authorization': 'Bearer ' + token}, \
                       data=payload)

results.json()

{'responseHeader': {'status': 0,
  'QTime': 44,
  'params': {'q': '*:*',
   'fl': 'bibcode,title',
   'start': '0',
   'internal_logging_params': 'X-Amzn-Trace-Id=Root=1-63dbf305-3a0bb5513f6d115d4e379a91',
   'fq': '{!bitset}',
   'rows': '2000',
   'wt': 'json'}},
 'response': {'numFound': 3,
  'start': 0,
  'docs': [{'bibcode': '1989LNP...334..242S',
    'title': ['The Optical and Radio Properties of X-Ray Selected Bl-Lacertae Objects']},
   {'bibcode': '1907AN....174...59.', 'title': ['Kleine Mitteilungen']},
   {'bibcode': '1908PA.....16..445.', 'title': ['Variable Stars']}]}}

## Query Parameters
All query parameters passed in the search URL must be UTF-8, URL-encoded strings. Due to the requirements of the authentication library used for validating requests, most non-ASCII characters that appear in the URL need to be encoded; for example, the double quote character (") should be encoded as %22. In most programming languages, such as Python, there are libraries to do this encoding for you, as shown in the examples in this notebook.

Query parameters are composed of a key and a value, separated by an equals (`=`) sign. More than one query parameter may be included in the URL; multiple parameters are separated by an ampersand (`&`). The query parameters are included at the end of the URL, preceded by a question mark (`?`). For example:

`https://api.adsabs.harvard.edu/v1/search/query?q=star&fl=title&rows=10`

#### q
**Required** The search query. This should be a UTF-8, URL-encoded string of <=1000 characters. `q` accepts both fields (`title:exoplanets`) and unfielded (`exoplanets`) searches.

#### rows 
The number of results to return. The default is 10 and the maximum is 2000.

#### start
The starting point for returned results, used for pagination. The default is 0. To return the next page of results, set `start` equal to the value of `start` from the previous request, plus the number of results returned in the previous request. For the default values, set `start=10` to return the second page of results.

#### fl
The list of fields to return. The value should be a comma separated list of field names, e.g. `fl=bibcode,author,title`. The default is the internal document id (`fl=id`). A non-exhaustive list of available fields is shown below.

#### fq
Filters the list of search results. The syntax is the same as that for the `q` parameter. Adding search parameters via the `fq` parameter can speed up search results, as it searches only the results returned by the search entered via the `q` parameter, not the entire index. This parameter may be used more than once in a single search URL.

#### sort
The sorting field and direction to be used when returning results. The format requires both the field to sort on (see the list below and the direction, either `asc` or `desc` (ascending or descending). For example, an appropriately formatted sort parameter is `sort=citation_count+desc`. The default sort method is the relevancy score as calculated by the search engine. Other useful fields to sort on may be `date`, `read_count`, `first_author`, or `bibcode`.

## Fields
This is a non-exhaustive list of fields available for searching via the API. A more comprehensive list is available in our [help pages](https://ui.adsabs.harvard.edu/help/search/comprehensive-solr-term-list). These fields can be used with the `fl` and `sort` parameters. 

Note: some fields, such as body, can be searched but not returned via `fl` or sorted on. Also, multivalued fields, such as author, cannot be used for sorting.

* `abstract` - the abstract of the record
* `ack` - the acknowledgements section of an article
* `aff` - an array of the authors' affiliations
* `alternate_bibcode` - list of alternate bibcodes for a single record; if a published record also has an arXiv version, the bibcode associated with the arXiv version is here
* `alternate_title` - list of alternate titles for a single record (usually if they are in multiple languages)
* `arxiv_class` - the arXiv class the pre-print was submitted to
* `author` - an array of the author names associated with the record
* `bibcode` - the canonical ADS bibcode identifier for this record
* `bibgroup` - the bibliographic groups that the bibcode has been associated with
* `bibstem` - the abbreviated name of the journal or publication, e.g., *ApJ*.
* `body`\* - the full text content of the article
* `citation_count` - number of citations the item has received
* `copyright` - the copyright applied to the article
* `data` - the list of sources that have data related to this bibcode
* `date` - the machine-readable version of `pubdate`, useful for sorting
* `database` - database the record is associated with (astronomy, physics, or general). By default, all three databases are searched; to limit to astronomy articles, add `fq=database:astronomy` to the URL
* `doi`-  the digital object identifier of the article
* `doctype` - the type of document it is (see [here](https://ui.adsabs.harvard.edu/help/search/search-syntax) for a list of doctypes)
* `first_author` - the first author of the article
* `grant` - the list of grant IDs and agencies noted within an article
* `id` - a **non-persistent** unique integer for this record, used for fast look-up of a document
* `identifier` - an array of alternative identifiers for the record. May contain alternate bibcodes, DOIs and/or arxiv ids.
* `indexstamp` - time at which the document was (last) indexed
* `issue` - issue the record appeared in
* `keyword` - an array of normalized and un-normalized keyword values associated with the record
* `lang`\* - the language of the article's title
* `orcid_pub` - ORCiD iDs supplied by publishers
* `orcid_user` - ORCiD iDs supplied by knonwn users in the ADS
* `orcid_other` - ORCiD iDs supplied by anonymous users in the ADS
* `page` - starting page
* `property` - an array of miscellaneous flags associated with the record (see [here](https://ui.adsabs.harvard.edu/help/search/search-syntax) for a list of properties
* `pub` - the canonical name of the publication the record appeared in
* `pubdate` - publication date in the form YYYY-MM-DD (DD value will always be "00"). This field is stored as a human-readable string, so is useful for being returned via `fl`, but not for sorting (see `date`)
* `read_count` - number of times the record has been viewed within in a 90-day windows (ads and arxiv)
* `title` - the title of the record
* `vizier` - the subject tags given to the article by VizieR
* `volume` - volume the record appeared in
* `year` - the year the article was published

\* *These fields are only indexed and so are only searchable - they are not stored or returned by the search end point, if requested*

## Examples
Some example requests are shown here. For more examples, see the `Examples` folder within this repository..

In [6]:
# search by bibcode, return the title
encoded_query = urlencode({"q": "bibcode:2011ApJ...737..103S",
                           "fl": "title"})
results = requests.get("https://api.adsabs.harvard.edu/v1/search/query?{}".format(encoded_query), \
                       headers={'Authorization': 'Bearer ' + token})

results.json()

{'responseHeader': {'status': 0,
  'QTime': 19,
  'params': {'q': 'bibcode:2011ApJ...737..103S',
   'fl': 'title',
   'start': '0',
   'internal_logging_params': 'X-Amzn-Trace-Id=Root=1-63dbf306-6f769deb4b0cdaca2d44b110',
   'rows': '10',
   'wt': 'json'}},
 'response': {'numFound': 1,
  'start': 0,
  'docs': [{'title': ['Measuring Reddening with Sloan Digital Sky Survey Stellar Spectra and Recalibrating SFD']}]}}

In [7]:
# search in all metadata fields for "black holes", restricted to the astronomy database; return a list of bibcodes
encoded_query = urlencode({"q": "black holes",
                           "fq": "database:astronomy",
                           "fl": "bibcode"})
results = requests.get("https://api.adsabs.harvard.edu/v1/search/query?{}".format(encoded_query), \
                       headers={'Authorization': 'Bearer ' + token})

results.json()

{'responseHeader': {'status': 0,
  'QTime': 1762,
  'params': {'q': 'black holes',
   'fl': 'bibcode',
   'start': '0',
   'internal_logging_params': 'X-Amzn-Trace-Id=Root=1-63dbf306-40906f1568cc13855bf47857',
   'fq': 'database:astronomy',
   'rows': '10',
   'wt': 'json'}},
 'response': {'numFound': 92420,
  'start': 0,
  'docs': [{'bibcode': '1973A&A....24..337S'},
   {'bibcode': '1975CMaPh..43..199H'},
   {'bibcode': '1974Natur.248...30H'},
   {'bibcode': '1973PhRvD...7.2333B'},
   {'bibcode': '2016PhRvL.116f1102A'},
   {'bibcode': '2013ARA&A..51..511K'},
   {'bibcode': '1977MNRAS.179..433B'},
   {'bibcode': '1976PhRvD..14..870U'},
   {'bibcode': '2023MNRAS.518.1057E'},
   {'bibcode': '1973blho.conf..343N'}]}}

In [8]:
# search in all metadata fields for "dark energy", filter by author, return a list of bibcodes and citation counts, 
# sorted by citation count
encoded_query = urlencode({"q": "'dark energy'",
                           "fq": "author:'Spergel, D.'",
                           "fl": "bibcode,citation_count",
                           "sort": "citation_count desc"
                          })
results = requests.get("https://api.adsabs.harvard.edu/v1/search/query?{}".format(encoded_query), \
                       headers={'Authorization': 'Bearer ' + token})

results.json()    

{'responseHeader': {'status': 0,
  'QTime': 1611,
  'params': {'q': "'dark energy'",
   'fl': 'bibcode,citation_count',
   'start': '0',
   'internal_logging_params': 'X-Amzn-Trace-Id=Root=1-63dbf308-0f5094965ae0d9047e58abd1',
   'fq': "author:'Spergel, D.'",
   'sort': 'citation_count desc',
   'rows': '10',
   'wt': 'json'}},
 'response': {'numFound': 109,
  'start': 0,
  'docs': [{'citation_count': 9711, 'bibcode': '2003ApJS..148..175S'},
   {'citation_count': 7601, 'bibcode': '2011ApJS..192...18K'},
   {'citation_count': 7053, 'bibcode': '2007ApJS..170..377S'},
   {'citation_count': 5016, 'bibcode': '2009ApJS..180..330K'},
   {'citation_count': 4498, 'bibcode': '2013ApJS..208...19H'},
   {'citation_count': 4416, 'bibcode': '2003ApJS..148....1B'},
   {'citation_count': 1740, 'bibcode': '2009ApJS..180..306D'},
   {'citation_count': 1607, 'bibcode': '2000PhRvL..84.3760S'},
   {'citation_count': 1583, 'bibcode': '2009ApJS..180..225H'},
   {'citation_count': 1449, 'bibcode': '2010MNRAS.

In [9]:
# limit a search to only refereed articles
encoded_query = urlencode({"q": "author:'Kurtz, M.'",
                           "fq": "property:refereed",
                           "fl": "title"
                          })
results = requests.get("https://api.adsabs.harvard.edu/v1/search/query?{}".format(encoded_query), \
                       headers={'Authorization': 'Bearer ' + token})

results.json()    

{'responseHeader': {'status': 0,
  'QTime': 298,
  'params': {'q': "author:'Kurtz, M.'",
   'fl': 'title',
   'start': '0',
   'internal_logging_params': 'X-Amzn-Trace-Id=Root=1-63dbf309-293e9f8c18ab409b3ce7df4c',
   'fq': 'property:refereed',
   'rows': '10',
   'wt': 'json'}},
 'response': {'numFound': 273,
  'start': 0,
  'docs': [{'title': ['The Updated Zwicky Catalog (UZC)']},
   {'title': ['RVSAO 2.0: Digital Redshifts and Radial Velocities']},
   {'title': ['The Disk-Outflow System around the Rare Young O-type Protostar W42-MME']},
   {'title': ["Hectospec, the MMT's 300 Optical Fiber-Fed Spectrograph"]},
   {'title': ['Discovery of an Unbound Hypervelocity Star in the Milky Way Halo']},
   {'title': ['A Galactic survey of radio jets from massive protostars']},
   {'title': ['Asteroseismology of the DOV Star PG 1159-035 with the Whole Earth Telescope']},
   {'title': ['Interstellar Complex Organic Molecules in SiO-traced Massive Outflows']},
   {'title': ['KOI-54: The Kepler Dis

In [10]:
# search for the phrase "transiting exoplanets", fetch 5 rows
encoded_query = urlencode({"q": "'transiting exoplanets'",
                           "fl": "bibcode",
                           "rows": 5,
                           "sort": "bibcode desc"
                          })
results = requests.get("https://api.adsabs.harvard.edu/v1/search/query?{}".format(encoded_query), \
                       headers={'Authorization': 'Bearer ' + token})

results.json()

{'responseHeader': {'status': 0,
  'QTime': 1338,
  'params': {'q': "'transiting exoplanets'",
   'fl': 'bibcode',
   'start': '0',
   'internal_logging_params': 'X-Amzn-Trace-Id=Root=1-63dbf30a-3dcba7901371607423016e73',
   'sort': 'bibcode desc',
   'rows': '5',
   'wt': 'json'}},
 'response': {'numFound': 9473,
  'start': 0,
  'docs': [{'bibcode': '2023yCat..36690117L'},
   {'bibcode': '2023nova.pres10368C'},
   {'bibcode': '2023lbep.book.....W'},
   {'bibcode': '2023asal.book..183Z'},
   {'bibcode': '2023arXiv230113708M'}]}}

In [11]:
# same, but return the next 5 rows
encoded_query = urlencode({"q": "'transiting exoplanets'",
                           "fl": "bibcode",
                           "rows": 5,
                           "start": 5,
                           "sort": "bibcode desc"
                          })
results = requests.get("https://api.adsabs.harvard.edu/v1/search/query?{}".format(encoded_query), \
                       headers={'Authorization': 'Bearer ' + token})

results.json()

{'responseHeader': {'status': 0,
  'QTime': 28,
  'params': {'q': "'transiting exoplanets'",
   'fl': 'bibcode',
   'start': '5',
   'internal_logging_params': 'X-Amzn-Trace-Id=Root=1-63dbf30b-1be8ee3d4e28eebe64893ab4',
   'sort': 'bibcode desc',
   'rows': '5',
   'wt': 'json'}},
 'response': {'numFound': 9473,
  'start': 5,
  'docs': [{'bibcode': '2023arXiv230113453I'},
   {'bibcode': '2023arXiv230113025C'},
   {'bibcode': '2023arXiv230112779P'},
   {'bibcode': '2023arXiv230110837L'},
   {'bibcode': '2023arXiv230110406L'}]}}

## Advanced Search Syntax
The `q` parameter supports both fielded and unfielded searches:
- `black holes`
- `title:exoplanets`

Use quotation marks to indicate phrase searching:
- `"black holes"`
- `title:"dark energy"`

Prepend terms with `+` or `-` to indicate inclusion or exclusion (note that after encoding spaces in the search string, the URL may include `++` or `+-`), or use Boolean operators (`AND`, `OR`, `NOT`):
- `"transiting exoplanets" -Kepler`
- `"transiting exoplanets" +JWST`

In [12]:
encoded_query = urlencode({"q": "'transiting exoplanets' +JWST",
                           "fl": "title",
                           "rows": 5,
                           "sort": "bibcode desc"
                          })
results = requests.get("https://api.adsabs.harvard.edu/v1/search/query?{}".format(encoded_query), \
                       headers={'Authorization': 'Bearer ' + token})

results.json()

{'responseHeader': {'status': 0,
  'QTime': 490,
  'params': {'q': "'transiting exoplanets' +JWST",
   'fl': 'title',
   'start': '0',
   'internal_logging_params': 'X-Amzn-Trace-Id=Root=1-63dbf30b-753e4c5c7669d7d417ed7732',
   'sort': 'bibcode desc',
   'rows': '5',
   'wt': 'json'}},
 'response': {'numFound': 972,
  'start': 0,
  'docs': [{'title': ['The Corgi of Exoplanets: Methane Mystery on HAT-P-18b']},
   {'title': ['The Little Book of Exoplanets']},
   {'title': ['The unusual M-dwarf Warm Jupiter TOI-1899~b: Refinement of orbital and planetary parameters']},
   {'title': ['A First Look at the JWST MIRI/LRS Phase Curve of WASP-43b']},
   {'title': ['A JWST transmission spectrum of a nearby Earth-sized exoplanet']}]}}

To filter by a publication date range, you can use either the `year` or the `pubdate` fields:
- `pubdate:[2013-07-00 TO *]`
- `pubdate:[2005-01 TO 2007-01]`
- `pubdate:2013-02`
- `year:2013`
- `year:[2012 TO 2013]`

The default search uses a boolean `AND` between terms, but you may use `OR` and `AND` in combination with `()` to create more complex queries.

Prefix queries (wildcards, `*`) are supported for most fields.

## Highlighting search terms
Snippets of text, with search terms highlighted, may be returned by a search query. Limits are in place to prevent scraping of our full-text holdings, as noted below, but more highlights can be returned via these API calls than are displayed in the search results on the website.

The following parameters control the highlights feature:
- `hl=true` - (required) turn on the highlighting feature
- `hl.fl` - (required) a comma-separated list of fields to return highlights for (e.g. title, abstract, or body)
- `hl.snippets` - number of highlighted snippets to return; the maximum is 4
- `hl.fragsize` - length of snippet to return; the maximum length is 100 characters. The snippet positioning is optimized to break on word and sentence boundaries, so the highlighted term will not always be centered in the snippet
- `hl.maxAnalyzedChars` - this parameter is only needed if the highlighted field is very long and the frequency of the search term is low. By default, the first 51,200 characters of a field are used for highlights. If the search is returning fewer snippets than expected, try increasing this value to capture more of the field. However, increasing this value unnecessarily will slow searching

Note: the highlights are returned in a separate section of the reponse than the results are, and are identified by the `id` of the result. The `id` is an internal identifier that is non-persistent with time, though will be consistent across a single response. So while it's not useful to store long term, the `id` should be included in the `fl` parameter in the request to match a given result to its highlights.

In [13]:
# similar to above, but with highlights returned
encoded_query = urlencode({"q": "body:('transiting exoplanets' +JWST)",
                           "fl": "title,id",
                           "rows": 3,
                           "sort": "score desc",
                           "hl": "true",
                           "hl.fl": "body",
                           "hl.snippets": 2,
                           "hl.fragsize": 100
                          })
results = requests.get("https://api.adsabs.harvard.edu/v1/search/query?{}".format(encoded_query), \
                       headers={'Authorization': 'Bearer ' + token})

results.json()

{'responseHeader': {'status': 0,
  'QTime': 1031,
  'params': {'hl.snippets': '2',
   'q': "body:('transiting exoplanets' +JWST)",
   'hl': 'true',
   'fl': 'title,id',
   'hl.fragsize': '100',
   'start': '0',
   'internal_logging_params': 'X-Amzn-Trace-Id=Root=1-63dbf30c-419b193d42bf96973cf4dbb6',
   'sort': 'score desc',
   'hl.fl': 'body',
   'rows': '3',
   'wt': 'json'}},
 'response': {'numFound': 3371,
  'start': 0,
  'docs': [{'id': '21464311',
    'title': ['Identification of carbon dioxide in an exoplanet atmosphere']},
   {'id': '1093212',
    'title': ['Transiting Exoplanet Survey Satellite (TESS)']},
   {'id': '21894614',
    'title': ['Early Release Science of the exoplanet WASP-39b with JWST NIRSpec PRISM']}]},
 'highlighting': {'21464311': {'body': [' 41. Beichman, C., et al. Observations of <em>Transiting</em> <em>Exoplanets</em> with the <em>James Webb Space Telescope</em>',
    'Identification of carbon dioxide in an <em>exoplanet</em> atmosphere <em>JWST</em> <em>Tr

## Downloading PDFs
PDFs may be downloaded using a GET request to the `link_gateway` endpoint. The URL to query consists of the base URL `https://ui.adsabs.harvard.edu/link_gateway/`, the bibcode to retrieve the PDF for, and an endpoint to indicate whether to download the arXiv PDF (`/EPRINT_PDF`) or the publisher PDF (`PUB_PDF`). 

The request will automatically redirect to the appropriate URL (arXiv or publisher), and the contents of the PDF will be available in the `.content` method of the response. Use the usual Python file writing functionality to write the PDF to file.

In [15]:
# download the arXiv PDF
results = requests.get("https://ui.adsabs.harvard.edu/link_gateway/1999ApJ...517..565P/EPRINT_PDF") 

# write the output to file
with open('/Users/klockhart/Downloads/output.pdf', 'wb') as f:
    f.write(results.content)
    
# show where the ADS URL redirected to
results.url

'https://arxiv.org/pdf/astro-ph/9812133.pdf'

In [ ]:
# download the publisher PDF
results = requests.get("https://ui.adsabs.harvard.edu/link_gateway/1998AJ....116.1009R/PUB_PDF")

# write the output to file
with open('/Users/klockhart/Downloads/output_pub.pdf', 'wb') as f:
    f.write(results.content)
    
# show where the ADS URL redirected to
results.url